Diese File dient der Erstellung und Speicherung eines AE. Die Auswertung des AE kann in R mittels der geeigneten Funktionen erfolgen. 

Input, welcher benötigt wird: exportFamaBliss_final30.csv, exportFamaBlisstimeonly30.csv, exportFamaBlissyieldsonly30.csv, mattimes30.csv und rownum30.csv

Output: export_yieldsx.xlsx, Encoder, Decoder, AE


In [ ]:
import csv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

In [ ]:
activ = 'tanh'
inputnodes = 30
bottleneck = 3

encoder_input_y = keras.Input(shape=(inputnodes), name="Spot-Rates")  
encoder_input_t = keras.Input(shape=(inputnodes), name="Laufzeit")

x = layers.concatenate([encoder_input_y, encoder_input_t], name="Zsmf.1")
#x = layers.Dense(62,activation=activ)(x)
x = layers.Dense(54,activation=activ)(x)
x = layers.Dense(42,activation=activ)(x)
x = layers.Dense(30,activation=activ)(x)
x = layers.Dense(18,activation=activ)(x)
x = layers.Dense(12,activation=activ)(x)
x = layers.Dense(6,activation=activ)(x)


encoder_output = layers.Dense(bottleneck,activation=activ)(x)

encoder = keras.Model(inputs=[encoder_input_y, encoder_input_t], outputs=encoder_output, name="Encoder")
#encoder.summary()


bottleneck_input = keras.Input(shape=(bottleneck,), name="Engpass") 
time_input = keras.Input(shape=(inputnodes), name="Laufzeit")


### Transforming the Data
## Encoder Output
#Coping the nodes of the encoder part next to each other. Extending the vector by one dim to a None x 3 x 1
reforminput1 = tf.keras.backend.expand_dims(bottleneck_input,axis=-1)
reforminput1 = tf.tile(reforminput1, multiples = [1,1,inputnodes])
# Shape of test2 is now Nonex3x30

## Time Input
encoder_input_t_2 = tf.keras.backend.expand_dims(time_input,axis=-1)
encoder_input_t_2 = tf.keras.backend.permute_dimensions(encoder_input_t_2, (0,2,1))
# Shape of encoder_input_t_2 is now None x 1 x inputnodes

## concatenate along axis = 1  

inputdecoder = layers.concatenate([reforminput1, encoder_input_t_2], name="Zsmf.2", axis=1)

## Decoder
x = tf.keras.backend.permute_dimensions(inputdecoder, (0,2,1))
#x = layers.Dense(4,activation=activ)(x)
#x = layers.Dense(4,activation=activ)(x)
x = layers.Dense(3,activation=activ)(x)
x = layers.Dense(2,activation=activ)(x)
x = layers.Dense(1,activation=activ)(x)
decoder_output = layers.Flatten(name="Spot-Rates")(x)


decoder = keras.Model(inputs=[bottleneck_input,time_input],outputs=decoder_output, name="Decoder")

encoded_values = encoder([encoder_input_y,encoder_input_t])
decoded_yields = decoder([encoded_values,encoder_input_t])


In [ ]:
# csv file name
filename = 'exportFamaBliss_final30.csv'
 
# initializing the titles and rows list
fields = []
x = []

with open(filename, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        x.append(row)

y = []
filename2 = 'exportFamaBlissyieldsonly30.csv'

with open(filename2, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        y.append(row)   
        
ti = []
filename3 = 'exportFamaBlisstimeonly30.csv'

with open(filename3, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        ti.append(row)   
        
#print(wichtig[0])
#len(wichtig)
#print(x[0])

for i in range(len(x)):
    x[i] = np.array_split(x[i], 2)
    


In [ ]:
# Instantiate an end-to-end model predicting both priority and department
model = keras.Model(inputs=[encoder_input_y, encoder_input_t],outputs=decoded_yields, name="Autoencoder")

model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

x1 = pad_sequences(x,maxlen=None,dtype='float64')
y1 = pad_sequences(y,maxlen=None,dtype='float64')
ti1 = pad_sequences(ti,maxlen=None,dtype='float64')

#scaler = StandardScaler()
#scaledy1 = scaler.fit_transform(y1)
#scaledti1 = scaler.fit_transform(ti1)

x_train, x_test, y_train, y_test = train_test_split(ti1, y1, test_size=0.15, random_state=1)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

with open('x_train.pickle', 'wb') as output:
    pickle.dump(x_train, output)
    
with open('x_test.pickle', 'wb') as output:
    pickle.dump(x_test, output)
    
with open('x_validation.pickle', 'wb') as output:
    pickle.dump(x_validation, output)
    
with open('y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
with open('y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
with open('y_validation.pickle', 'wb') as output:
    pickle.dump(y_validation, output)
    
    
history = model.fit(x=[y_train,x_train], y=y_train, epochs=3000, batch_size=32, validation_data=([y_validation,x_validation], y_validation))
#history = model.fit(x=[y_train,x_train], y=y_train, epochs=1000, batch_size=64)




In [ ]:
plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

with open('history.pickle', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)


In [ ]:
plt.plot(history.history['loss'][350:])
#plt.plot(history.history['val_loss'][350:])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate([y_test,x_test],y_test, batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict([y_test,x_test])
#predictions = scaler.inverse_transform(predictions)
print("predictions shape:", predictions.shape)


In [ ]:
array111 = encoder.predict([y1,ti1])
import pandas as pd

df = pd.DataFrame(array111, columns = ['x1','x2','x3'])
df.to_excel (r'export_dataframe1.xlsx', index = False, header=True)


In [ ]:
tiexport = []
filename4 = 'mattimes30.csv'

with open(filename4, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        tiexport.append(row)   
        
        
rownums = []
filename5 = 'rownum30.csv'

with open(filename5, newline='') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        rownums.append(row)   
        
rownums = pad_sequences(rownums,maxlen=None,dtype='float64')

In [ ]:
arrayfinal = []
tiexport1 = pad_sequences(tiexport,maxlen=None,dtype='float64')

for i in range(len(array111)):
    if i==0:
        a = 0
        b = int(rownums[0][0])
        tiexport1_new = tiexport1[a:b]
        input1 = []
        for _ in range(len(tiexport1_new)):
            input1.append(array111[i])
        
        input1 = pad_sequences(input1,maxlen=None,dtype='float64')
        array222 = decoder.predict([input1,tiexport1_new])
        
        arrayfinal.append(array222)
    else:
        a = int(sum(rownums[0:i])[0])
        b = int(sum(rownums[0:(i+1)])[0])

        tiexport1_new = tiexport1[a:b]
        input1 = []
        for _ in range(len(tiexport1_new)):
            input1.append(array111[i])
        
        input1 = pad_sequences(input1,maxlen=None,dtype='float64')
        array222 = decoder.predict([input1,tiexport1_new])
        
        arrayfinal.append(array222)
        

In [ ]:
arraytestt = []
for i in range(len(arrayfinal)):
    for j in range(len(arrayfinal[i])):
        arraytestt.append(arrayfinal[i][j])

arrayfin = pd.DataFrame(arraytestt)
arrayfin.to_excel (r'export_yields0.xlsx', index = False, header=True)

In [ ]:
keras.utils.plot_model(model, "AutoenconderModel0.png", show_shapes=True)
keras.utils.plot_model(encoder, "Encoder0.png", show_shapes=True)
keras.utils.plot_model(decoder, "Decoder0.png", show_shapes=True)

In [ ]:
encoder.save('encoder0')
decoder.save('decoder0')
model.save('autoencoder0')

In [ ]:
reconstructed_model = keras.models.load_model("autoencoder0")

# Let's check:
np.testing.assert_allclose(
    model.predict([y_test,x_test]), reconstructed_model.predict([y_test,x_test])
)

